## This notebook is to merge two counts data: APNIC and union

Let me explain how two CSV files were created in emerald machine.

## APNIC data: org_counts_apnic.csv

+ Raw data files are located at: emerald:/data/bulk_apnic
+ The processing can be done with: emerald:/var/opt/spam2/mapping/apnic/run_apnic.sh

Briefly explaining the processing steps, "run_get_ip_nb_as_count_from_apnic.py" does the followings:
+ first extract a gzipped file (YYYY-MM-DD-apnic.RPSL.db.gz) from above raw data folder
+ in gzipped file, find blocks where "descr" field includes our focal companies' names 
+ if a block has "aut-num" field, then it is AS-related information
+ if a block has "inetnum" field, then it is netblock information. With the netblock size, we can get the IP count
+ all these AS, netblock, IP counts are stored as JSON files in emerald:/var/opt/spam2/mapping/apnics/data
+ it takes an hour or so to process one gzipped file

Once all the JSON files are ready, then "make_csv_from_apnic_data.py" does the followings:
+ read JSON files from emerald:/var/opt/spam2/mapping/apnics/data
+ create "org_counts_apnic.csv"


## Union data: org_counts_union.csv

+ Raw data files are located at: emerald:/data/bulk_cymru/raw
+ "emerald:/var/opt/spam2/mapping/parse_asn_netblock_mapping_from_cymru.py" proceses the raw cymru files to get netblock-to-ASN mappings every day
+ the resulting mappings are stored in "emerald:/var/opt/spam2/mapping/union/pickle" as Python pickle files
+ "emerald:/var/opt/spam2/mapping/apnic/run_get_ip_nb_as_count_from_union.py" read those Picke files to generate "org_counts_union.csv" file


## Now we can two CSV files are ready. Time to merge them!

I used a few principles here:
+ Find the months that both CSV files are covering
+ For each month, get the larger IP/Netblock/AS counts from the two because each has partially coverage
+ For example, "union" data cannot cover the cases where an org has netblocks but not ASN
+ "apnic" data is created with string matching, it cannot cover the cases where netblocks' names are abbreviated
+ For example, "China National Petroleum Corporation" was used in AS description but "CNPC" was used for netblocks

## OK now let's merge!

In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv('./org_counts_apnic.csv')
df2 = pd.read_csv('./org_counts_union.csv')

In [3]:
months1 = set()
for col in df1.columns:
    if col.count('_') == 0:
        continue
    months1.add(col.split('_')[-1])
print months1

set(['201601', '201603', '201602', '201605', '201604', '201607', '201606', '201609', '201608', '201403', '201404', '201701', '201702', '201703', '201704', '201705', '201502', '201503', '201507', '201504', '201505', '201612', '201610', '201611'])


In [4]:
months2 = set()
for col in df2.columns:
    if col.count('_') == 0:
        continue
    months2.add(col.split('_')[-1])
print months2

set(['201612', '201701', '201702', '201703', '201704', '201610', '201705', '201601', '201603', '201602', '201605', '201604', '201606', '201609', '201608', '201611'])


In [5]:
inter = months1 & months2
month_inter = sorted(list(inter))
print month_inter

['201601', '201602', '201603', '201604', '201605', '201606', '201608', '201609', '201610', '201611', '201612', '201701', '201702', '201703', '201704', '201705']


We see that both union and APNIC data cover from 2016/01 to 2017/05, with an exception of 2016/07. 

With the assumption that AS/netblock mappings are not super dynamic, let's replace the missed 2016/07 records with 2016/06.

In [6]:
from pprint import pprint

metrics = dict()
metrics['ip'] = []
metrics['nb'] = []
metrics['as'] = []

# construct dictionary for later merged dataframe
counts = dict()
for month in inter:
    counts['ip{}'.format(month)] = []
    counts['nb{}'.format(month)] = []
    counts['as{}'.format(month)] = []
    
for index, row in df1.iterrows():
    #print row['DossierID'], row['org']
    this_counts = dict()
    this_counts['ip'] = []
    this_counts['nb'] = []
    this_counts['as'] = []
    for month in month_inter:
        #print month,
        for metric in metrics:
            apnic_count = row['{}_apnic_{}'.format(metric, month)]
            union_count = df2.ix[index]['{}_union_{}'.format(metric, month)]
            larger_count = max(apnic_count, union_count)
            this_counts[metric].append(larger_count)
            
            counts['{}{}'.format(metric, month)].append(larger_count)
            
            #if apnic_count ==0 and union_count == 0:
            #    #metrics[metric] += 1
            #print apnic_count, union_count, (apnic_count==union_count),
        #print
    for metric in this_counts:
        if sum(this_counts[metric]) == 0:
            metrics[metric].append(str(row['DossierID']) + ' ' + row['org'])
        #print
        

pprint(metrics['ip'])
print len(metrics['ip'])

['1004 Magna Automotive Technology and Service(Shanghai)Co Ltd',
 '1010 Founder Group',
 '1016 YunGang Technology Co Ltd',
 '1021 Beijing Capital Public Information Platform',
 '1023 Changhong IT information Products Co Ltd',
 '1025 Beijing Swifton inc',
 '1028 Beijing Haves Cinda Sci-Tech Development Co Ltd',
 '1033 Beijing Sinnet Technology Co Ltd',
 '1038 BeiJing Shocom Telecom Co Ltd',
 '1041 United e-Communicaiton (Beijing) S&T Ltd',
 '1042 Beijing Kaixinren Information Technology Company',
 '1045 BEIJING ZHONGGUANCUN SOFTWARE PARK DEVELOPMENT Co Ltd',
 '1047 BEIJING SHENZHOU GREATWALL COMMUNICATION',
 '1059 CITIC Ltd',
 '1069 Guangdong Hutong Broadband Network Co Ltd',
 '1097 Jiangxi Broadcasting and TV information Network',
 '1112 China Broadcasting TV Net',
 '1172 Guangzhou Tenmark Networks Technology Ltd',
 '1174 Easynet Global Services Asia',
 '1179 SHANGHAI Guangdian Electronics Group Co Ltd',
 '1185 Fumeiti Technology Co',
 '1196 GOIP-AULA-LIMITED',
 '1207 FoShan SHITONG In

Above is the list of 178 organizations without any netblock/IP information. At least we made a big improvement from 339 zero-IP orgs from previous data. Hopefully we have enough samples now!

## now let's make the merged dataframe

In [7]:
new_df = pd.read_excel('./for_randomization_170426.xlsx')

metrics = ['ip', 'nb', 'as']
print month_inter

['201601', '201602', '201603', '201604', '201605', '201606', '201608', '201609', '201610', '201611', '201612', '201701', '201702', '201703', '201704', '201705']


In [8]:
for metric in metrics:
    for month in month_inter:
        new_df['{}{}'.format(metric, month)] = counts['{}{}'.format(metric, month)]
        # let's fill in 2016/07 with 2016/06
        if month == '201606':
            new_df['{}201607'.format(metric)] = counts['{}{}'.format(metric, month)]

new_df.head()

,DossierID,org,IsChinese,hsic,Industry Description,Country,hasFacebook,hasTwitter,hasWeibo,hasWeChat,...,pv201612,pv201701,pv201702,pv201703,ip201704,ip201705,nb201704,nb201705,as201704,as201705
0,1001,LiaoHe Oilfield Telecommunication Company,Y,631200,Web portals,CN,0,0,0,1,...,0,0,0,0,49152,49152,5,5,1,1
1,1002,Beijing Expo Cloud Technology Co Ltd,Y,631100,"data processing, hosting and related activities",CN,0,0,0,0,...,6,0,0,0,512,512,2,2,1,1
2,1003,Shenzhen Aosida Communication Co Ltd,Y,611000,Telecommunications network operation,CN,0,0,0,0,...,0,0,0,0,183296,183296,14,14,1,1
3,1004,Magna Automotive Technology and Service(Shangh...,Y,290000,Body assembly of motor vehicles,CN,0,0,1,1,...,0,0,0,0,0,0,0,0,1,1
4,1005,Guizhou Wing Cloud High Technology Ltd,Y,631100,"Data processing, hosting and related activities",CN,0,0,0,0,...,0,0,0,0,67584,67584,3,3,2,2


In [9]:
new_df.describe()

,DossierID,hsic,hasFacebook,hasTwitter,hasWeibo,hasWeChat,hasTencentWeibo,ip201512,ip201601,ip201602,...,pv201612,pv201701,pv201702,pv201703,ip201704,ip201705,nb201704,nb201705,as201704,as201705
count,1262.000000,1262.000000,1262.000000,1262.000000,1262.000000,1262.000000,1262.000000,1262.0,1.262000e+03,1.262000e+03,...,1262.000000,1262.000000,1262.000000,1262.000000,1.262000e+03,1.262000e+03,1262.000000,1262.000000,1262.000000,1262.000000
mean,1910.603803,626104.491284,0.480190,0.342314,0.267829,0.240095,0.126783,0.0,4.793416e+05,4.757112e+05,...,233.512678,62.649762,195.183043,117.457211,4.833362e+05,4.808875e+05,35.610143,35.899366,1.673534,1.676704
std,587.160233,119032.960562,0.499805,0.474672,0.443003,0.427310,0.332862,0.0,5.279871e+06,5.281042e+06,...,2370.397172,887.293733,2882.801924,1705.388801,5.385209e+06,5.345345e+06,238.915555,242.535118,4.166013,4.167403
min,1001.000000,50000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,1.000000,1.000000
25%,1401.500000,611000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,5.120000e+02,2.720000e+02,...,0.000000,0.000000,0.000000,0.000000,2.560000e+02,2.560000e+02,1.000000,1.000000,1.000000,1.000000
50%,1825.500000,620200.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,2.048000e+03,2.048000e+03,...,0.000000,0.000000,0.000000,0.000000,2.048000e+03,2.048000e+03,3.000000,3.000000,1.000000,1.000000
75%,2531.500000,639100.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.0,1.638400e+04,1.638400e+04,...,0.000000,0.000000,0.000000,0.000000,1.708800e+04,1.728000e+04,12.000000,12.000000,1.000000,1.000000
max,2922.000000,960299.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.344774e+08,1.351965e+08,...,57849.000000,21765.000000,69434.000000,43868.000000,1.397320e+08,1.380352e+08,5516.000000,5555.000000,119.000000,119.000000


In [10]:
new_df.to_excel('./for_randomization_170509.xlsx')

After this Excel file was created, I did some manual column reordering for beautification.